## Sprocket Central Pty Ltd Company Customers Recommendation  Data wrangling

### About the Dataset

**Sprocket Central Pty Ltd**, a medium size bikes & cycling accessories organisation which has a large dataset relating to its customers, but their team is unsure how to effectively analyse it to help optimise its marketing strategy. 

The client provided us with 3 datasets:

- Customer Demographic

- Customer Addresses

- Transactions data in the past 3 months

### Objective of the report

Our objective is to review the data quality for the three datasets to ensure that they are ready for our analysis , we will start wrangling and fixing each one of them seperately and then join them together into one dataset

In this Phase #1 we will improve the quality of Sprocket Central Pty Ltd’s data through:

- Data wrangling and Cleaning

- Features engineering

- Joinning the datasets together

### 1. Data Wrangling & cleaning

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
import calendar

from datetime import datetime
%matplotlib inline


## Setting max displayed rows to 500, in order to display the full output of any command 
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

### Transactions Dataset

In [2]:
#Importing The transactions dataset
Transactions=pd.read_excel('Transactions.xlsx')

In [3]:
# checking the data first rows
Transactions.head(1)

,Note: The data and information in this document is reflective of a hypothetical situation and client. This document is to be used for KPMG Virtual Internship purposes only.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date


We need to drop the first unnamed row that is created in the dataframe and adjust the first row as header,we will create a function that will remove the header and exchange it with the first row of the data so we can use it in the other dataframes.

In [4]:
#creating the function
def titlehead(x):
    x.rename(columns=x.iloc[0], inplace = True)
    x.drop([0], inplace = True)

In [5]:
# applying the function on the data sets
titlehead(Transactions)

In [6]:
# checking for results
Transactions.head(1)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
1,1,2,2950,2017-02-25 00:00:00,False,Approved,Solex,Standard,medium,medium,71.49,53.62,41245


In [7]:
# Checking dataset info
Transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 1 to 20000
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   transaction_id           20000 non-null  object
 1   product_id               20000 non-null  object
 2   customer_id              20000 non-null  object
 3   transaction_date         20000 non-null  object
 4   online_order             19640 non-null  object
 5   order_status             20000 non-null  object
 6   brand                    19803 non-null  object
 7   product_line             19803 non-null  object
 8   product_class            19803 non-null  object
 9   product_size             19803 non-null  object
 10  list_price               20000 non-null  object
 11  standard_cost            19803 non-null  object
 12  product_first_sold_date  19803 non-null  object
dtypes: object(13)
memory usage: 2.1+ MB


The general information about the dataframe points out to several problems:

- **list price,standard cost** columns are stored as object, where in fact it should be numeric.
- The **Transactions date** column should be converted into datetime columns not object
- **Transaction date** column should have year,month name,day,day name columns
- **online_order,brand, product line,product class,product size,standard cost and product_first_sold_date** have missing values.

### Features Engineering

In [8]:
# How many missing points in each variable
count_missing_Transactions = Transactions.isnull().sum()
percent_missing_Transactions = round(Transactions.isnull().sum()/len(Transactions) * 100, 1)
missing_Transactions = pd.concat([count_missing_Transactions, percent_missing_Transactions], axis = 1)
missing_Transactions.columns = ["Missing (count)", "Missing (%)"]
missing_Transactions

,Missing (count),Missing (%)
transaction_id,0,0.0
product_id,0,0.0
customer_id,0,0.0
transaction_date,0,0.0
online_order,360,1.8
order_status,0,0.0
brand,197,1.0
product_line,197,1.0
product_class,197,1.0
product_size,197,1.0


**How to deal with Missing Data?**

There are many strategies to fill missing data:

1. Fill with the mean (better used in case of continous variables without outliers)
2. Fill with the median (better used in case of continous variables with outliers)
3. Fill with the mode (better used in case of categorical variables)
4. Drop the entire variable if the number of missing points is too large
5. Drop the rows containing null values if it's not going to affect our analysis

**Based on the above strategies:**

- **online order:** has just 1.8% missing values, se we will fill them with the mode (the most frequent data point)
- **brand, product line,product class,product size,standard cost and product_first_sold_date** have 1% missing values , they seem to have the same missing data pattern, we will remove the rows contatining their null values since it is just 1% and will not affect our analysis

In [9]:
# filling the online order column with the mode value 
Transactions['online_order'] = Transactions['online_order'].fillna(Transactions['online_order'].mode()[0])

In [10]:
# Removing other data rows with nul values
Transactions.dropna(axis=0,inplace=True)

In [11]:
# checking results after removing null values

Transactions.isnull().sum()

transaction_id             0
product_id                 0
customer_id                0
transaction_date           0
online_order               0
order_status               0
brand                      0
product_line               0
product_class              0
product_size               0
list_price                 0
standard_cost              0
product_first_sold_date    0
dtype: int64

In [12]:
# ammending column data types
Transactions['list_price']=pd.to_numeric(Transactions['list_price'])
Transactions['standard_cost']=pd.to_numeric(Transactions['standard_cost'])
Transactions['transaction_date']=pd.to_datetime(Transactions['transaction_date'])

In [13]:
# Checking for the changes info
Transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19803 entries, 1 to 20000
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           19803 non-null  object        
 1   product_id               19803 non-null  object        
 2   customer_id              19803 non-null  object        
 3   transaction_date         19803 non-null  datetime64[ns]
 4   online_order             19803 non-null  bool          
 5   order_status             19803 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               19803 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n

In [14]:
# checking for duplicated rows

Transactions[Transactions.duplicated()]

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date


There are no duplicated rows in the data

we will now check the consistency of the values in each categorical column in the dataset,we will create a list that has all the categorical columns from the data set

In [15]:
# Collecting the categorical columns into  list
cat_col=[]
for x in Transactions.dtypes.index:
    if Transactions.dtypes[x]=='object':
        cat_col.append(x)
cat_col

['transaction_id',
 'product_id',
 'customer_id',
 'order_status',
 'brand',
 'product_line',
 'product_class',
 'product_size',
 'product_first_sold_date']

listing all the unique values inside each categorical column of the recently created list

In [16]:
#checking for duplicated values in the categorical columns nd the accuracy of the values
for col in cat_col:
    print(col)
    print(Transactions[col].unique())
    print()
    print('*******')
    print()

transaction_id
[1 2 3 ... 19998 19999 20000]

*******

product_id
[2 3 37 88 78 25 22 15 67 12 5 61 35 16 79 33 54 27 82 89 64 19 72 91 1 99
 0 92 14 44 76 46 55 66 81 86 32 77 96 6 47 94 93 60 28 4 38 56 58 50 80
 87 84 21 31 62 17 73 45 49 95 18 70 26 39 36 98 75 42 20 24 53 65 29 11
 10 7 41 9 69 90 97 100 74 71 34 57 23 51 59 63 40 8 13 30 48 68 83 43 52
 85]

*******

customer_id
[2950 3120 402 ... 2764 3168 3464]

*******

order_status
['Approved' 'Cancelled']

*******

brand
['Solex' 'Trek Bicycles' 'OHM Cycles' 'Norco Bicycles' 'Giant Bicycles'
 'WeareA2B']

*******

product_line
['Standard' 'Road' 'Mountain' 'Touring']

*******

product_class
['medium' 'low' 'high']

*******

product_size
['medium' 'large' 'small']

*******

product_first_sold_date
[41245 41701 36361 36145 42226 39031 34165 39915 33455 38216 40784 42172
 34527 34586 38193 37873 38206 33888 37337 36334 42145 42404 34079 41047
 42560 42710 41922 37539 42688 38991 38647 37874 34996 33549 38693 37668
 41533 41009 

**product_first_sold_date** column seem to have a wrong date format which we will ammend

In [17]:
#converting dates to datetime in pandas
def convert_to_datetime(num):
    dt = datetime.fromordinal(datetime(1900, 1, 1).toordinal() + num - 2)
    return dt

In [18]:
#applying the function on the date transactions
Transactions['product_first_sold_date']=Transactions['product_first_sold_date'].apply(convert_to_datetime)
Transactions.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
1,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02
2,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03
3,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20
4,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16
5,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10


We will create new variables as follows:

1. Transaction year
2. Transaction month name
3. Transaction day
4. Transaction day name

In [19]:
# Changing transactions date column into transaction year,month,day,day_name columns
Transactions['Transaction_year']=Transactions['transaction_date'].dt.year
Transactions['Transaction_month']=Transactions['transaction_date'].dt.month_name()
Transactions['Transaction_day']=Transactions['transaction_date'].dt.day
Transactions['day_of_the_week']=Transactions['transaction_date'].dt.day_name()

# converting the transaction year , day columnes into string columns
Transactions['Transaction_year']=Transactions['Transaction_year'].astype(str)
Transactions['Transaction_day']=Transactions['Transaction_day'].astype(str)

**Transactions dataset is clean and ready for analysis**

In [ ]:
### Customer demographics dataset

In [21]:
#Importing customer demographics dataset
Customer_demographic=pd.read_excel('Customer demographic.xlsx')

In [22]:
#checking dataset head
Customer_demographic.head(2)

,Note: The data and information in this document is reflective of a hypothetical situation and client. This document is to be used for KPMG Virtual Internship purposes only.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
1,1,Laraine,Medendorp,F,93,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11


Making the first row as header using the (titlehead) function created

In [23]:
titlehead(Customer_demographic)

In [24]:
#checking results
Customer_demographic.head(2)

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
1,1,Laraine,Medendorp,F,93,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11
2,2,Eli,Bockman,Male,81,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16


In [25]:
Customer_demographic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 1 to 4000
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   customer_id                          4000 non-null   object
 1   first_name                           4000 non-null   object
 2   last_name                            3875 non-null   object
 3   gender                               4000 non-null   object
 4   past_3_years_bike_related_purchases  4000 non-null   object
 5   DOB                                  3913 non-null   object
 6   job_title                            3494 non-null   object
 7   job_industry_category                3344 non-null   object
 8   wealth_segment                       4000 non-null   object
 9   deceased_indicator                   4000 non-null   object
 10  default                              3698 non-null   object
 11  owns_car                             4000 n

The general information about the dataframe points out to several problems:

1. **past_3_years_bike_related_purchases,tenure** is stored as object, where in fact it has no numeric meaning.
2. **DOB** is stored as object where it should be converted to datetime
3. **last_name,DOB,job_title,job_industry_category,default,tenure** has null values

### Features Engineering

In [26]:
# How many missing points in each variable
count_missing_Customer_demographic = Customer_demographic.isnull().sum()
percent_missing_Customer_demographic = round(Customer_demographic.isnull().sum()/len(Customer_demographic) * 100, 1)
missing_train = pd.concat([count_missing_Customer_demographic, percent_missing_Customer_demographic], axis = 1)
missing_train.columns = ["Missing (count)", "Missing (%)"]
missing_train

,Missing (count),Missing (%)
customer_id,0,0.0
first_name,0,0.0
last_name,125,3.1
gender,0,0.0
past_3_years_bike_related_purchases,0,0.0
DOB,87,2.2
job_title,506,12.6
job_industry_category,656,16.4
wealth_segment,0,0.0
deceased_indicator,0,0.0


**Dealing with Missing Data**

all of the columns are categorical columns so either i will fill in the null values with the mode of these columns or i will drop the column entirley if the missing percentage is high

1. **last_name:** has just 3.1% missing values, so we fill nulls with mode
2. **DOB:** has 2.2% so we fill nulls with mode
3. **job_title:** has 12.6% we will fill these values with mode
4. **job_industry_category:** has 16.4% so we fill nulls with mode
5. **default:** has 7.6% so we fill nulls with mode
6. **Tenure:** has 2.2% missing from the column so we will fill it with mean

In [27]:
#Filling in columns with mode 
Customer_demographic['last_name'] = Customer_demographic['last_name'].fillna(Customer_demographic['last_name'].mode()[0])
Customer_demographic['DOB'] = Customer_demographic['DOB'].fillna(Customer_demographic['DOB'].mode()[0])
Customer_demographic['job_title'] = Customer_demographic['job_title'].fillna(Customer_demographic['job_title'].mode()[0])
Customer_demographic['job_industry_category'] = Customer_demographic['job_industry_category'].fillna(Customer_demographic['job_industry_category'].mode()[0])
Customer_demographic['default'] = Customer_demographic['default'].fillna(Customer_demographic['default'].mode()[0])
Customer_demographic['tenure'] = Customer_demographic['tenure'].fillna(Customer_demographic['tenure'].mean())

In [28]:
# Changing column values types
Customer_demographic['past_3_years_bike_related_purchases']=pd.to_numeric(Customer_demographic['past_3_years_bike_related_purchases'])
Customer_demographic['tenure']=pd.to_numeric(Customer_demographic['tenure'])
Customer_demographic['DOB']=pd.to_datetime(Customer_demographic['DOB'])

In [29]:
# checking for duplicated rows 
Customer_demographic[Customer_demographic.duplicated()]

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure


There are no duplicate rows

In [30]:
# Collecting the categorical columns into  list

cat_col=[]
for x in Customer_demographic.dtypes.index:
    if Customer_demographic.dtypes[x]=='object':
        cat_col.append(x)
cat_col

['customer_id',
 'first_name',
 'last_name',
 'gender',
 'job_title',
 'job_industry_category',
 'wealth_segment',
 'deceased_indicator',
 'default',
 'owns_car']

In [31]:
#checking for consistency of the values in each categorical column in the data

for col in cat_col:
    print(col)
    print(Customer_demographic[col].unique())
    print()
    print('*******')
    print()

customer_id
[1 2 3 ... 3998 3999 4000]

*******

first_name
['Laraine' 'Eli' 'Arlin' ... 'Stephie' 'Rusty' 'Sarene']

*******

last_name
['Medendorp' 'Bockman' 'Dearle' ... 'Halgarth' 'Woolley' 'Oldland']

*******

gender
['F' 'Male' 'Female' 'U' 'Femal' 'M']

*******

job_title
['Executive Secretary' 'Administrative Officer' 'Recruiting Manager'
 'Business Systems Development Analyst' 'Senior Editor' 'Media Manager I'
 'Senior Quality Engineer' 'Nuclear Power Engineer' 'Developer I'
 'Account Executive' 'Junior Executive' 'Media Manager IV'
 'Sales Associate' 'Professor' 'Geological Engineer' 'Project Manager'
 'Safety Technician I' 'Research Assistant I' 'Accounting Assistant III'
 'Editor' 'Research Nurse' 'Safety Technician III' 'Staff Accountant III'
 'Legal Assistant' 'Product Engineer' 'Information Systems Manager'
 'VP Quality Control' 'Social Worker' 'Senior Cost Accountant'
 'Assistant Media Planner' 'Payment Adjustment Coordinator' 'Food Chemist'
 'Accountant III' 'Director 

IT seems that there are repeated values in **Gender** column and  inconsistent values in **default** column

In [32]:
# Ammending values in gender column
Customer_demographic.replace({'gender':{'F':'Female','Femal':'Female','M':'Male'}},inplace=True)

In [33]:
# removing U value from gender column
Customer_demographic=Customer_demographic[Customer_demographic.gender!='U']

In [34]:
# drop default columns
Customer_demographic.drop('default',axis=1,inplace=True)

we will create features engineering for some columns

1. Changing DOB column into Age column
2. Dropping DOB after conversion

In [35]:
# This function converts given date to age
def from_dob_to_age(born):
    today = dt.date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [36]:
#applying the function on the DOB column
Customer_demographic['Age']=Customer_demographic['DOB'].apply(lambda x: from_dob_to_age(x))

In [37]:
# dropping DOB column
Customer_demographic.drop('DOB',axis=1,inplace=True)

In [38]:
Customer_demographic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3912 entries, 1 to 4000
Data columns (total 12 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   customer_id                          3912 non-null   object 
 1   first_name                           3912 non-null   object 
 2   last_name                            3912 non-null   object 
 3   gender                               3912 non-null   object 
 4   past_3_years_bike_related_purchases  3912 non-null   int64  
 5   job_title                            3912 non-null   object 
 6   job_industry_category                3912 non-null   object 
 7   wealth_segment                       3912 non-null   object 
 8   deceased_indicator                   3912 non-null   object 
 9   owns_car                             3912 non-null   object 
 10  tenure                               3912 non-null   float64
 11  Age                           

In [39]:
#checking results
Customer_demographic.head(2)

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age
1,1,Laraine,Medendorp,Female,93,Executive Secretary,Health,Mass Customer,N,Yes,11.0,67
2,2,Eli,Bockman,Male,81,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,40


**Customer demographic dataset is clean and ready for analysis**

### Customer address dataset

In [40]:
# Importing dataset
Customer_address=pd.read_excel('Customer address.xlsx')

In [41]:
#checking first rows 
Customer_address.head(2)

,Note: The data and information in this document is reflective of a hypothetical situation and client. This document is to be used for KPMG Virtual Internship purposes only.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,customer_id,address,postcode,state,country,property_valuation
1,1,060 Morning Avenue,2016,New South Wales,Australia,10


In [42]:
#using the function to remove unnamed row and adjust first row to header
titlehead(Customer_address)

In [43]:
#checking for results
Customer_address.head(2)

,customer_id,address,postcode,state,country,property_valuation
1,1,060 Morning Avenue,2016,New South Wales,Australia,10
2,2,6 Meadow Vale Court,2153,New South Wales,Australia,10


In [44]:
#checking dataset info
Customer_address.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3999 entries, 1 to 3999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   object
 1   address             3999 non-null   object
 2   postcode            3999 non-null   object
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   object
dtypes: object(6)
memory usage: 218.7+ KB


In [45]:
#checking for duplicated rows
Customer_address[Customer_address.duplicated()]

,customer_id,address,postcode,state,country,property_valuation


There are no duplicated rows in the dataset

In [46]:
# appending the categorical columns into a list
cat_col=[]
for x in Customer_address.dtypes.index:
    if Customer_address.dtypes[x]=='object':
        cat_col.append(x)
cat_col

['customer_id',
 'address',
 'postcode',
 'state',
 'country',
 'property_valuation']

In [47]:
#checking for values inconsistencies in the categorical columns

for col in cat_col:
    print(col)
    print(Customer_address[col].unique())
    print()
    print('*******')
    print()

customer_id
[1 2 4 ... 4001 4002 4003]

*******

address
['060 Morning Avenue' '6 Meadow Vale Court' '0 Holy Cross Court' ...
 '87 Crescent Oaks Alley' '8194 Lien Street' '320 Acker Drive']

*******

postcode
[2016 2153 4211 2448 3216 2210 2650 2023 3044 4557 3799 2760 2428 3331
 3058 2135 2233 2444 4413 4740 3218 4868 4116 2519 2756 2170 4005 4127
 2535 4726 4805 2212 2207 3350 3931 4018 4670 2480 3212 2322 3198 2047
 4503 2145 4350 2142 2029 2127 4280 2166 2046 2026 2765 2749 2745 3752
 2217 2230 3046 3183 2070 2213 4208 2774 4133 4213 4170 2073 3561 2330
 3977 2204 2086 2226 2075 2038 2069 2100 3133 3070 2777 2125 2766 2209
 2160 2065 2150 2168 4171 2155 2517 2096 2750 2093 3204 4300 3844 4226
 3192 2485 2097 4817 2516 3074 4508 4405 2452 2205 4132 4744 2158 3201
 4178 2112 2033 4401 3186 4017 2315 2285 2219 4509 2759 2747 2227 2025
 3191 3025 2263 2154 2119 3016 4113 2032 4352 3020 2116 3057 2099 3749
 2148 3145 2021 2333 2783 2280 4120 3638 2074 2880 2430 4560 2088 2220
 3031 2250

**Country:** Country column represents one value only which is Australia so i will remove it since it will not be helpful in my analysis

General data information states that data types are consistent and there are no null values

**Customer address dataset is clean and ready for analysis**

Since the three dataset are related to each other in which the first dataset is the transactions made by customers and the second dataset is the demographics of customers who made the transactions and third dataset is the addresses of these customers , we can conduct an inner join between them throught the customer id primary key column in the first dataset and customer id foregin keys columns in demographics and addresses datasets.

In [48]:
# joining Transactions with customer demographics datasets
Transactions_demographics=Transactions.merge(Customer_demographic,on='customer_id',how='inner')
Transactions_demographics.head(2)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,Transaction_year,Transaction_month,Transaction_day,day_of_the_week,first_name,last_name,gender,past_3_years_bike_related_purchases,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02,2017,February,25,Saturday,Kristos,Anthony,Male,19,Software Engineer I,Financial Services,Mass Customer,N,Yes,10.0,66
1,11065,1,2950,2017-10-16,False,Approved,Giant Bicycles,Standard,medium,medium,1403.50,954.82,2003-02-07,2017,October,16,Monday,Kristos,Anthony,Male,19,Software Engineer I,Financial Services,Mass Customer,N,Yes,10.0,66


In [49]:
# merging customersdemographics & transactions with addresses dataset into a CTA datasets (Customers,Transcations,Addresses)
CTA=Transactions_demographics.merge(Customer_address,on='customer_id',how='inner')

In [50]:
#checking the final general information of the three datasets together
CTA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19327 entries, 0 to 19326
Data columns (total 33 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   transaction_id                       19327 non-null  object        
 1   product_id                           19327 non-null  object        
 2   customer_id                          19327 non-null  object        
 3   transaction_date                     19327 non-null  datetime64[ns]
 4   online_order                         19327 non-null  bool          
 5   order_status                         19327 non-null  object        
 6   brand                                19327 non-null  object        
 7   product_line                         19327 non-null  object        
 8   product_class                        19327 non-null  object        
 9   product_size                         19327 non-null  object        
 10  list_price

In [51]:
#checking final data first rows
CTA.head(2)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,Transaction_year,Transaction_month,Transaction_day,day_of_the_week,first_name,last_name,gender,past_3_years_bike_related_purchases,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age,address,postcode,state,country,property_valuation
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02,2017,February,25,Saturday,Kristos,Anthony,Male,19,Software Engineer I,Financial Services,Mass Customer,N,Yes,10.0,66,984 Hoepker Court,3064,VIC,Australia,6
1,11065,1,2950,2017-10-16,False,Approved,Giant Bicycles,Standard,medium,medium,1403.50,954.82,2003-02-07,2017,October,16,Monday,Kristos,Anthony,Male,19,Software Engineer I,Financial Services,Mass Customer,N,Yes,10.0,66,984 Hoepker Court,3064,VIC,Australia,6


now we will Export data to csv file to use it in our next juyter notebook for phase 2 of the project which is Exploratory data analysis

In [52]:
#Exporting data to csv file
CTA.to_csv('CTA_wrangling.csv')